### Investigate how well LoRA Adapter works for PerMU

In [54]:
import os

directory_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/experiment/TOFU/llama2-7b'

model_paths = []

# List all entries in the directory
for entry in os.listdir(directory_path):
    full_path = os.path.join(directory_path, entry)
    # Check if the entry is a directory
    if os.path.isdir(full_path):
        model_paths.append(entry)

In [55]:
import pandas as pd
import json

import os

models_list = []  # Initialize as a list to store DataFrames

for mp in model_paths:
    eval_path = os.path.join(directory_path, mp, 'eval_results')  # Use os.path.join for safe path concatenation
    eval_csv_path = os.path.join(eval_path, 'eval.csv')  # Use os.path.join for safe path concatenation
    adapter_path = os.path.join(directory_path, mp, 'adapter_config.json')  # Use os.path.join for safe path concatenation
    if os.path.exists(adapter_path): 
        with open(adapter_path, 'r') as f:
            data = json.load(f)
        # Extract 'r' and 'alpha' from the JSON data
        r = data.get('r', None)  # Default to None if 'r' is not found
        alpha = data.get('lora_alpha', None)  # Default to None if 'alpha' is not found
    else:
        r = None
        alpha = None

    if os.path.exists(eval_csv_path): 
        eval_df  = pd.read_csv(eval_csv_path)
        eval_df['model'] = mp  # Add model column
        eval_df['r'] = r
        eval_df['alpha'] = alpha
        models_list.append(eval_df)  # Append the DataFrame to the list


models_df = pd.concat(models_list, ignore_index=True)

In [56]:
models_df.loc[models_df['model'] == 'FullPerMU_E5_B4_G3_lr2e-5_W1', 'r'] = 'Full'
models_df.loc[models_df['model'] == 'FullPerMU_E5_B4_G3_lr2e-5_W1', 'alpha'] = 'Full'


In [57]:
models_df.columns

Index(['ROUGE Real Authors', 'Prob. Real Authors', 'Truth Ratio Real Authors',
       'ROUGE Real World', 'Prob. Real World', 'Truth Ratio Real World',
       'ROUGE Retain', 'Prob. Retain', 'Truth Ratio Retain', 'ROUGE Forget',
       'Prob. Forget', 'Truth Ratio Forget', 'ROUGE Forget Rephrase',
       'Prob. Forget Rephrase', 'Truth Ratio Forget Rephrase',
       'ROUGE Retain Rephrase', 'Prob. Retain Rephrase',
       'Truth Ratio Retain Rephrase', 'Model Utility', 'Fluency Retain',
       'Fluency Real Authors', 'Fluency Real World', 'Fluency Forget',
       'Fluency Forget Rephrase', 'Fluency Retain Rephrase', 'Method',
       'Submitted By', 'model', 'r', 'alpha'],
      dtype='object')

In [60]:
models_df[['r','alpha','ROUGE Forget','Prob. Forget','Truth Ratio Forget','ROUGE Retain','Prob. Retain']].sort_values(by='ROUGE Forget')

,r,alpha,ROUGE Forget,Prob. Forget,Truth Ratio Forget,ROUGE Retain,Prob. Retain
0,Full,Full,0.201400,0.055003,0.814520,0.695517,0.774193
1,256,512,0.331686,0.282246,0.716789,0.752455,0.800025
5,64,256,0.659229,0.725987,0.607645,0.931128,0.932313
4,256,256,0.681055,0.737396,0.604429,0.937420,0.934949
2,128,256,0.682897,0.731295,0.605706,0.931191,0.932106
3,16,32,0.949886,0.991656,0.539787,0.982276,0.988570
